# 🎯 Embeddings e Representações: Transformando Palavras em Números Mágicos

**Módulo 5 de 13 - Introdução à LLMs**

Por: Pedro Nunes Guth 🚀

---

Bora mergulhar no mundo dos embeddings! Se você chegou até aqui, já entendeu sobre tokens e tokenização no módulo anterior. Agora vamos descobrir como transformar essas palavrinhas em números que fazem sentido para os computadores!

**O que você vai aprender:**
- O que são embeddings (spoiler: não é comida!)
- Como representar palavras em vetores
- Word2Vec, GloVe e embeddings contextuais
- Como os Transformers usam embeddings
- Na prática: implementando embeddings do zero!

In [ ]:
# Bora começar importando as bibliotecas que vamos usar!
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Configurações visuais
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🎉 Bibliotecas carregadas! Bora para os embeddings!")

## 🤔 Tá, mas o que são Embeddings?

Imagina que você é um tradutor tentando explicar para um alienígena o que significa "cachorro". Como você faria isso?

Você poderia dizer:
- "É um animal que late"
- "Tem 4 patas"
- "É peludo"
- "É amigo dos humanos"

**Embeddings fazem exatamente isso!** Eles pegam uma palavra e a representam através de números que capturam suas características e significados.

### Por que precisamos disso?

Computadores não entendem "cachorro" ou "gato". Eles só entendem números! Então precisamos converter palavras em vetores numéricos que preservem o significado semântico.

**Dica do Pedro:** Think of embeddings como o "DNA numérico" das palavras! 🧬

## 📊 Visualizando o Conceito de Embeddings

Vamos começar com um exemplo visual para entender como palavras podem ser representadas em um espaço vetorial:

In [ ]:
# Vamos criar embeddings simples para algumas palavras
# Imagine que temos apenas 2 dimensões: "animal" e "tamanho"

palavras = ['cachorro', 'gato', 'elefante', 'rato', 'leão', 'peixe']

# Embeddings 2D simples [dimensão_animal, dimensão_tamanho]
embeddings_2d = {
    'cachorro': [0.8, 0.6],   # Muito animal, tamanho médio
    'gato': [0.9, 0.4],       # Muito animal, pequeno
    'elefante': [0.7, 0.9],   # Animal, muito grande
    'rato': [0.6, 0.1],       # Pouco animal, muito pequeno
    'leão': [0.85, 0.8],      # Muito animal, grande
    'peixe': [0.5, 0.3]       # Pouco animal (diferente), pequeno
}

# Plotando os embeddings
fig, ax = plt.subplots(figsize=(10, 8))

for palavra, (x, y) in embeddings_2d.items():
    ax.scatter(x, y, s=200, alpha=0.7)
    ax.annotate(palavra, (x, y), xytext=(5, 5), textcoords='offset points', 
                fontsize=12, fontweight='bold')

ax.set_xlabel('Dimensão: Animalidade', fontsize=14)
ax.set_ylabel('Dimensão: Tamanho', fontsize=14)
ax.set_title('Embeddings 2D Simplificados\n(Palavras no Espaço Vetorial)', fontsize=16)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()

print("🎯 Repare como palavras similares ficam próximas no espaço!")

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-05_img_01.png)

## 🔄 A Evolução dos Embeddings

Os embeddings evoluíram muito ao longo do tempo. Vamos entender essa jornada:

```mermaid
graph TD
    A[One-Hot Encoding] --> B[Word2Vec]
    B --> C[GloVe]
    C --> D[FastText]
    D --> E[Embeddings Contextuais]
    E --> F[BERT/GPT Embeddings]
    
    A --> A1["Problema: Vetores enormes e esparsos"]
    B --> B1["Solução: Vetores densos e semânticos"]
    C --> C1["Melhoria: Considera estatísticas globais"]
    D --> D1["Inovação: Considera subpalavras"]
    E --> E1["Revolução: Contexto importa!"]
    F --> F1["Estado da arte: Transformers"]
```

### 1. One-Hot Encoding (O Início Ingênuo)

Era como dar um CPF único para cada palavra, mas sem nenhuma relação semântica:
- Cachorro: [1, 0, 0, 0, 0]
- Gato: [0, 1, 0, 0, 0]

**Problema:** Vetores gigantes e sem significado semântico!

In [ ]:
# Exemplo de One-Hot Encoding
vocabulario = ['cachorro', 'gato', 'elefante', 'rato', 'leão']

def one_hot_encoding(palavra, vocab):
    """Cria one-hot encoding para uma palavra"""
    vector = [0] * len(vocab)
    if palavra in vocab:
        index = vocab.index(palavra)
        vector[index] = 1
    return vector

# Testando
print("One-Hot Encoding:")
for palavra in vocabulario[:3]:
    encoding = one_hot_encoding(palavra, vocabulario)
    print(f"{palavra}: {encoding}")

# Calculando similaridade (spoiler: sempre será 0!)
cachorro_vec = one_hot_encoding('cachorro', vocabulario)
gato_vec = one_hot_encoding('gato', vocabulario)

similarity = cosine_similarity([cachorro_vec], [gato_vec])[0][0]
print(f"\n🤔 Similaridade entre 'cachorro' e 'gato': {similarity}")
print("Isso não faz sentido! Eles são ambos animais domésticos!")

## 🚀 Word2Vec: A Revolução dos Embeddings

Em 2013, o Google lançou o Word2Vec e mudou tudo! A ideia genial foi:

**"Me diga com quem andas e te direi quem és"**

Palavras que aparecem em contextos similares têm significados similares!

### Como funciona?

Existem duas abordagens:
1. **CBOW (Continuous Bag of Words):** Prediz a palavra central dado o contexto
2. **Skip-gram:** Prediz o contexto dada a palavra central

**Dica do Pedro:** É como aprender português ouvindo conversas. Se você sempre ouve "cachorro" perto de "late", "peludo", "animal", você aprende que eles estão relacionados! 🐕

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-05_img_02.png)

In [ ]:
# Vamos implementar um Word2Vec simplificado do zero!
# (Versão didática para entender o conceito)

class SimpleWord2Vec:
    def __init__(self, vector_size=100, window=2, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.vocab = {}
        self.word_vectors = {}
        
    def build_vocab(self, sentences):
        """Constrói o vocabulário a partir das sentenças"""
        word_count = {}
        
        # Conta frequência das palavras
        for sentence in sentences:
            for word in sentence.split():
                word = word.lower().strip('.,!?')
                word_count[word] = word_count.get(word, 0) + 1
        
        # Filtra palavras por frequência mínima
        self.vocab = {word: idx for idx, (word, count) in enumerate(word_count.items()) 
                     if count >= self.min_count}
        
        # Inicializa vetores aleatórios
        np.random.seed(42)
        for word in self.vocab:
            self.word_vectors[word] = np.random.normal(0, 0.1, self.vector_size)
            
        print(f"📚 Vocabulário construído com {len(self.vocab)} palavras!")
        
    def get_vector(self, word):
        """Retorna o vetor de uma palavra"""
        return self.word_vectors.get(word.lower(), None)
    
    def similarity(self, word1, word2):
        """Calcula similaridade cosseno entre duas palavras"""
        vec1 = self.get_vector(word1)
        vec2 = self.get_vector(word2)
        
        if vec1 is None or vec2 is None:
            return 0
            
        return cosine_similarity([vec1], [vec2])[0][0]

# Testando nosso Word2Vec simples
sentences = [
    "o cachorro late muito alto",
    "o gato mia baixinho",
    "cachorros e gatos são animais domésticos",
    "animais domésticos precisam de cuidados",
    "o elefante é um animal gigante",
    "elefantes vivem na savana africana",
    "gatos gostam de pescar peixes",
    "cachorros adoram brincar no parque"
]

model = SimpleWord2Vec(vector_size=50)
model.build_vocab(sentences)

print(f"\n🔍 Palavras no vocabulário: {list(model.vocab.keys())[:10]}")

## 📈 Visualizando Embeddings em Ação

Agora vamos ver como os embeddings capturam relações semânticas:

In [ ]:
# Vamos criar embeddings mais realistas para demonstração
np.random.seed(42)

# Embeddings pré-definidos que capturam relações semânticas
embeddings_realistas = {
    # Animais domésticos
    'cachorro': np.array([0.8, 0.6, 0.9, 0.7, 0.5]),
    'gato': np.array([0.9, 0.5, 0.8, 0.6, 0.4]),
    
    # Animais selvagens
    'leão': np.array([0.7, 0.9, 0.6, 0.8, 0.9]),
    'elefante': np.array([0.6, 0.8, 0.5, 0.9, 0.8]),
    
    # Objetos
    'carro': np.array([0.2, 0.1, 0.3, 0.2, 0.1]),
    'casa': np.array([0.1, 0.2, 0.1, 0.3, 0.2]),
    
    # Comidas
    'pizza': np.array([0.3, 0.2, 0.4, 0.1, 0.3]),
    'hamburguer': np.array([0.4, 0.3, 0.5, 0.2, 0.4])
}

# Calculando matriz de similaridades
palavras = list(embeddings_realistas.keys())
vectors = list(embeddings_realistas.values())

similarity_matrix = cosine_similarity(vectors)

# Criando heatmap de similaridades
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(similarity_matrix, cmap='RdYlBu_r', aspect='auto')

# Configurações do plot
ax.set_xticks(range(len(palavras)))
ax.set_yticks(range(len(palavras)))
ax.set_xticklabels(palavras, rotation=45, ha='right')
ax.set_yticklabels(palavras)

# Adicionando valores na matriz
for i in range(len(palavras)):
    for j in range(len(palavras)):
        text = ax.text(j, i, f'{similarity_matrix[i, j]:.2f}',
                      ha="center", va="center", color="black", fontweight='bold')

ax.set_title('Matriz de Similaridade dos Embeddings\n(Quanto mais vermelho, mais similar)', 
             fontsize=14, pad=20)

# Colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Similaridade Cosseno', rotation=270, labelpad=20)

plt.tight_layout()
plt.show()

print("🎯 Repare como:")
print("- Cachorro e gato têm alta similaridade (ambos animais domésticos)")
print("- Leão e elefante também são similares (animais selvagens)")
print("- Pizza e hambúrguer têm similaridade (ambos comidas)")
print("- Animais vs objetos têm baixa similaridade")

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-05_img_03.png)

## 🧠 Embeddings Contextuais: A Nova Era

Aqui vem a parada mais liiinda! Os embeddings contextuais (como os usados em BERT, GPT, etc.) não dão apenas UM vetor por palavra. Eles criam vetores diferentes dependendo do CONTEXTO!

### Por que isso é revolucionário?

Pense na palavra "banco":
- "Vou ao **banco** sacar dinheiro" (instituição financeira)
- "Sentei no **banco** da praça" (assento)

Embeddings tradicionais dariam o mesmo vetor para ambos. Embeddings contextuais dão vetores diferentes!

**Dica do Pedro:** É como ter um tradutor que entende o contexto da conversa, não só palavras isoladas! 🎯

In [ ]:
# Simulando embeddings contextuais
# (Na prática, isso vem de modelos como BERT/GPT)

def gerar_embedding_contextual(palavra, contexto, dimensoes=5):
    """
    Simula como embeddings contextuais funcionam
    """
    np.random.seed(hash(contexto) % 1000)  # Seed baseada no contexto
    
    # Base embedding da palavra
    base_embedding = np.random.normal(0, 0.5, dimensoes)
    
    # Modificação contextual
    context_modification = np.random.normal(0, 0.3, dimensoes)
    
    return base_embedding + context_modification

# Testando com a palavra "banco" em contextos diferentes
contextos = {
    "financeiro": "Vou ao banco sacar dinheiro para pagar as contas",
    "assento": "Sentei no banco da praça para descansar um pouco",
    "dados": "O banco de dados do sistema está funcionando perfeitamente"
}

embeddings_contextuais = {}

print("🎭 Diferentes embeddings para 'banco' conforme o contexto:\n")

for tipo, contexto in contextos.items():
    embedding = gerar_embedding_contextual("banco", contexto)
    embeddings_contextuais[tipo] = embedding
    print(f"{tipo.upper()}:")
    print(f"Contexto: {contexto}")
    print(f"Embedding: {embedding.round(3)}\n")

# Calculando similaridades
tipos = list(embeddings_contextuais.keys())
print("🔍 Similaridades entre os diferentes contextos:")

for i, tipo1 in enumerate(tipos):
    for tipo2 in tipos[i+1:]:
        sim = cosine_similarity([embeddings_contextuais[tipo1]], 
                               [embeddings_contextuais[tipo2]])[0][0]
        print(f"{tipo1} vs {tipo2}: {sim:.3f}")

## 🔧 Como os Transformers Usam Embeddings

Lembra dos Transformers que estudamos no Módulo 3? Eles usam três tipos de embeddings:

```mermaid
graph LR
    A[Token] --> B[Token Embedding]
    A --> C[Position Embedding] 
    A --> D[Segment Embedding]
    
    B --> E[Soma]
    C --> E
    D --> E
    
    E --> F[Input para Transformer]
    
    B --> B1["Significado da palavra"]
    C --> C1["Posição na sequência"]
    D --> D1["Qual sentença (BERT)"]
```

### 1. Token Embeddings
O significado semântico da palavra/token

### 2. Position Embeddings  
A posição da palavra na sequência (muito importante!)

### 3. Segment Embeddings
Usado em modelos como BERT para distinguir diferentes sentenças

In [ ]:
# Implementando os três tipos de embeddings usados em Transformers

class TransformerEmbeddings:
    def __init__(self, vocab_size=1000, embed_dim=128, max_length=512):
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.max_length = max_length
        
        # Inicializando embeddings aleatórios (na prática, são aprendidos)
        np.random.seed(42)
        self.token_embeddings = np.random.normal(0, 0.1, (vocab_size, embed_dim))
        self.position_embeddings = np.random.normal(0, 0.1, (max_length, embed_dim))
        self.segment_embeddings = np.random.normal(0, 0.1, (2, embed_dim))  # 2 segmentos
        
    def get_embeddings(self, token_ids, positions, segment_ids):
        """
        Combina os três tipos de embeddings
        """
        # Token embeddings
        token_emb = self.token_embeddings[token_ids]
        
        # Position embeddings
        pos_emb = self.position_embeddings[positions]
        
        # Segment embeddings
        seg_emb = self.segment_embeddings[segment_ids]
        
        # Soma tudo (como fazem os Transformers)
        final_embeddings = token_emb + pos_emb + seg_emb
        
        return {
            'token': token_emb,
            'position': pos_emb, 
            'segment': seg_emb,
            'final': final_embeddings
        }

# Testando
embedder = TransformerEmbeddings(embed_dim=8)  # Dimensão pequena para visualizar

# Exemplo: "O gato subiu no telhado"
token_ids = [1, 45, 234, 12, 567]  # IDs dos tokens
positions = [0, 1, 2, 3, 4]        # Posições
segment_ids = [0, 0, 0, 0, 0]      # Todos da mesma sentença

embeddings = embedder.get_embeddings(token_ids, positions, segment_ids)

print("🔧 Embeddings dos Transformers:")
print(f"\nToken embeddings shape: {embeddings['token'].shape}")
print(f"Position embeddings shape: {embeddings['position'].shape}")
print(f"Segment embeddings shape: {embeddings['segment'].shape}")
print(f"\nFinal embeddings (primeiro token): {embeddings['final'][0].round(3)}")

## 📏 Métricas e Avaliação de Embeddings

Como sabemos se nossos embeddings são bons? Existem várias métricas:

### 1. Similaridade Cosseno
Mede o ângulo entre dois vetores (0 = ortogonais, 1 = idênticos)

### 2. Analogias
"Rei" - "Homem" + "Mulher" ≈ "Rainha"

### 3. Clustering
Palavras similares devem formar clusters

In [ ]:
# Implementando avaliação de embeddings

def avaliar_analogias(embeddings_dict):
    """
    Testa analogias do tipo: A - B + C ≈ D
    """
    analogias = [
        ('rei', 'homem', 'mulher', 'rainha'),
        ('paris', 'frança', 'brasil', 'brasilia'),
        ('grande', 'maior', 'pequeno', 'menor')
    ]
    
    print("🧮 Testando analogias (conceitual):")
    
    for a, b, c, d_esperado in analogias:
        if all(word in embeddings_dict for word in [a, b, c]):
            # Calcula: A - B + C
            resultado = (embeddings_dict[a] - embeddings_dict[b] + embeddings_dict[c])
            
            # Encontra a palavra mais próxima
            max_sim = -1
            palavra_mais_proxima = None
            
            for palavra, embedding in embeddings_dict.items():
                if palavra not in [a, b, c]:  # Exclui palavras da entrada
                    sim = cosine_similarity([resultado], [embedding])[0][0]
                    if sim > max_sim:
                        max_sim = sim
                        palavra_mais_proxima = palavra
            
            print(f"{a} - {b} + {c} = {palavra_mais_proxima} (esperado: {d_esperado})")
            print(f"Similaridade: {max_sim:.3f}\n")

# Criando embeddings de exemplo para teste
embeddings_teste = {
    'rei': np.array([0.8, 0.9, 0.7, 0.6]),
    'rainha': np.array([0.9, 0.8, 0.6, 0.7]), 
    'homem': np.array([0.7, 0.5, 0.8, 0.4]),
    'mulher': np.array([0.8, 0.4, 0.7, 0.5]),
    'paris': np.array([0.3, 0.8, 0.9, 0.2]),
    'brasilia': np.array([0.4, 0.7, 0.8, 0.3]),
    'frança': np.array([0.2, 0.9, 0.8, 0.1]),
    'brasil': np.array([0.3, 0.8, 0.7, 0.2])
}

avaliar_analogias(embeddings_teste)

## 🎨 Visualização em 2D com PCA

Embeddings geralmente têm centenas de dimensões. Para visualizar, usamos PCA (Análise de Componentes Principais) para reduzir para 2D:

In [ ]:
# Criando embeddings mais ricos para visualização
np.random.seed(42)

# Categorias de palavras com embeddings similares dentro da categoria
categorias = {
    'animais': ['cachorro', 'gato', 'leão', 'tigre', 'elefante'],
    'comidas': ['pizza', 'hamburguer', 'sushi', 'lasanha', 'salada'],
    'veiculos': ['carro', 'moto', 'avião', 'barco', 'bicicleta'],
    'cores': ['vermelho', 'azul', 'verde', 'amarelo', 'roxo']
}

# Gerando embeddings agrupados por categoria
embeddings_categorias = {}
cores_plot = ['red', 'blue', 'green', 'orange']
cores_mapa = {}

for i, (categoria, palavras) in enumerate(categorias.items()):
    # Centro da categoria
    centro = np.random.normal(i*3, 1, 10)  # 10 dimensões
    
    for palavra in palavras:
        # Adiciona ruído ao centro para criar cluster
        embedding = centro + np.random.normal(0, 0.5, 10)
        embeddings_categorias[palavra] = embedding
        cores_mapa[palavra] = cores_plot[i]

# Aplicando PCA para reduzir para 2D
palavras = list(embeddings_categorias.keys())
vectors = list(embeddings_categorias.values())

pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(vectors)

# Plotando
fig, ax = plt.subplots(figsize=(12, 10))

# Plot por categoria
for i, (categoria, palavras_cat) in enumerate(categorias.items()):
    indices = [palavras.index(p) for p in palavras_cat]
    x_coords = [embeddings_2d[idx][0] for idx in indices]
    y_coords = [embeddings_2d[idx][1] for idx in indices]
    
    ax.scatter(x_coords, y_coords, c=cores_plot[i], label=categoria.title(), 
              s=200, alpha=0.7, edgecolors='black', linewidth=1)
    
    # Anotações
    for idx, palavra in zip(indices, palavras_cat):
        ax.annotate(palavra, (embeddings_2d[idx][0], embeddings_2d[idx][1]), 
                   xytext=(5, 5), textcoords='offset points', 
                   fontsize=10, fontweight='bold')

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} da variância)', fontsize=12)
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} da variância)', fontsize=12)
ax.set_title('Visualização de Embeddings em 2D\n(Redução dimensional com PCA)', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"🎯 PCA explica {sum(pca.explained_variance_ratio_):.2%} da variância total")
print("Repare como palavras da mesma categoria ficam agrupadas!")

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-05_img_04.png)

## 🔗 Conectando com os Próximos Módulos

Os embeddings que aprendemos hoje são fundamentais para entender:

### Módulo 6 - Tipos de Modelos
- Como diferentes arquiteturas (BERT, GPT, T5) usam embeddings
- Embeddings especializados para tarefas específicas

### Módulo 7 - Treinamento
- Como os embeddings são aprendidos durante o treinamento
- Técnicas de fine-tuning de embeddings

### Módulo 8 - Prompting
- Como prompts são convertidos em embeddings
- Engenharia de prompts no espaço vetorial

**Dica do Pedro:** Embeddings são como o "sistema circulatório" dos LLMs - eles transportam significado por toda a rede! 🩸

## 🏋️ Exercício Prático 1: Construindo seu Próprio Sistema de Embeddings

Agora é sua vez! Vamos implementar um sistema completo de embeddings:

In [ ]:
# EXERCÍCIO 1: Complete o código abaixo

class MeuSistemaEmbeddings:
    def __init__(self, dimensoes=50):
        self.dimensoes = dimensoes
        self.vocab = {}
        self.embeddings = {}
        self.treinado = False
    
    def treinar(self, textos):
        """
        TODO: Implemente o treinamento dos embeddings
        
        Passos:
        1. Construa o vocabulário a partir dos textos
        2. Inicialize embeddings aleatórios para cada palavra
        3. (Bonus) Implemente uma versão simples de co-ocorrência
        """
        pass  # Substitua por sua implementação
    
    def obter_embedding(self, palavra):
        """
        TODO: Retorne o embedding de uma palavra
        """
        pass  # Substitua por sua implementação
    
    def similaridade(self, palavra1, palavra2):
        """
        TODO: Calcule a similaridade cosseno entre duas palavras
        """
        pass  # Substitua por sua implementação
    
    def palavras_similares(self, palavra, top_k=3):
        """
        TODO: Encontre as k palavras mais similares
        """
        pass  # Substitua por sua implementação

# Dados para teste
textos_treino = [
    "cachorro late muito alto no quintal",
    "gato mia baixinho pela manhã", 
    "carro vermelho anda na estrada",
    "bicicleta azul está na garagem",
    "pizza deliciosa com queijo",
    "hamburguer saboroso com batata"
]

# Teste seu sistema aqui
# meu_sistema = MeuSistemaEmbeddings()
# meu_sistema.treinar(textos_treino)
# print(meu_sistema.similaridade('cachorro', 'gato'))

print("🎯 Complete o código acima e teste seu sistema de embeddings!")

## 🧪 Exercício Prático 2: Análise de Bias em Embeddings

Embeddings podem carregar preconceitos dos dados de treino. Vamos investigar isso:

In [ ]:
# EXERCÍCIO 2: Detectando bias em embeddings

# Embeddings com bias simulado
embeddings_com_bias = {
    'médico': np.array([0.8, 0.7, 0.9, 0.6]),
    'enfermeira': np.array([0.3, 0.8, 0.4, 0.9]),
    'engenheiro': np.array([0.9, 0.6, 0.8, 0.5]),
    'professor': np.array([0.5, 0.7, 0.6, 0.8]),
    'homem': np.array([0.9, 0.5, 0.8, 0.4]),
    'mulher': np.array([0.2, 0.9, 0.3, 0.8]),
    'forte': np.array([0.8, 0.4, 0.9, 0.3]),
    'delicada': np.array([0.2, 0.8, 0.3, 0.9])
}

def analisar_bias(embeddings_dict, profissoes, generos, adjetivos):
    """
    TODO: Analise associações entre profissões, gêneros e adjetivos
    
    Calcule:
    1. Similaridade entre profissões e gêneros
    2. Similaridade entre gêneros e adjetivos
    3. Identifique possíveis vieses
    """
    
    print("🔍 ANÁLISE DE BIAS EM EMBEDDINGS")
    print("=" * 40)
    
    # TODO: Implemente sua análise aqui
    
    # Exemplo de estrutura:
    # for profissao in profissoes:
    #     for genero in generos:
    #         sim = calcular_similaridade(profissao, genero)
    #         print(f"{profissao} <-> {genero}: {sim:.3f}")
    
    pass

# Teste sua análise
profissoes = ['médico', 'enfermeira', 'engenheiro', 'professor']
generos = ['homem', 'mulher']
adjetivos = ['forte', 'delicada']

# analisar_bias(embeddings_com_bias, profissoes, generos, adjetivos)

print("\n💡 Dicas para sua implementação:")
print("- Use similaridade cosseno")
print("- Procure por associações problemáticas")
print("- Pense em como mitigar esses vieses")

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-05_img_05.png)

## 📊 Comparando Diferentes Tipos de Embeddings

Vamos fazer uma comparação final entre os principais tipos de embeddings:

In [ ]:
# Comparação final entre tipos de embeddings

import pandas as pd

# Criando tabela comparativa
comparacao = {
    'Tipo': ['One-Hot', 'Word2Vec', 'GloVe', 'FastText', 'BERT/GPT'],
    'Ano': [1990, 2013, 2014, 2017, 2018],
    'Dimensões': ['Vocabulário', '50-300', '50-300', '100-300', '768-1024'],
    'Contextual': ['Não', 'Não', 'Não', 'Não', 'Sim'],
    'Subpalavras': ['Não', 'Não', 'Não', 'Sim', 'Sim'],
    'Vantagem Principal': [
        'Simplicidade',
        'Semântica capturada',
        'Estatísticas globais',
        'Palavras raras/OOV',
        'Contexto dinâmico'
    ],
    'Desvantagem': [
        'Sem semântica',
        'Sem contexto',
        'Computacionalmente caro',
        'Treino complexo',
        'Muito pesado'
    ]
}

df_comparacao = pd.DataFrame(comparacao)
print("📊 EVOLUÇÃO DOS EMBEDDINGS")
print("=" * 50)
print(df_comparacao.to_string(index=False))

# Gráfico da evolução temporal
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Timeline
anos = df_comparacao['Ano'].values
tipos = df_comparacao['Tipo'].values
cores = ['red', 'orange', 'yellow', 'lightgreen', 'darkgreen']

ax1.scatter(anos, range(len(tipos)), c=cores, s=200, alpha=0.7)
for i, (ano, tipo) in enumerate(zip(anos, tipos)):
    ax1.annotate(tipo, (ano, i), xytext=(10, 0), 
                textcoords='offset points', fontsize=10, fontweight='bold')

ax1.set_xlabel('Ano', fontsize=12)
ax1.set_ylabel('Evolução', fontsize=12)
ax1.set_title('Timeline dos Embeddings', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.set_yticks([])

# Complexidade vs Performance (conceitual)
complexidade = [1, 3, 4, 5, 8]
performance = [1, 6, 7, 8, 10]

ax2.scatter(complexidade, performance, c=cores, s=200, alpha=0.7)
for i, tipo in enumerate(tipos):
    ax2.annotate(tipo, (complexidade[i], performance[i]), 
                xytext=(5, 5), textcoords='offset points', 
                fontsize=10, fontweight='bold')

ax2.set_xlabel('Complexidade', fontsize=12)
ax2.set_ylabel('Performance', fontsize=12)
ax2.set_title('Complexidade vs Performance', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎯 Lições principais:")
print("• Embeddings evoluíram de simples para sofisticados")
print("• Trade-off entre simplicidade e performance")
print("• Contexto fez toda a diferença!")

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-05_img_06.png)

## 🎓 Resumo do Módulo: Embeddings Dominados!

Liiindo! Você chegou ao final do Módulo 5! Vamos recapitular o que aprendemos:

### 🧠 Conceitos Principais
- **Embeddings**: Representações numéricas que capturam significado semântico
- **Evolução**: De One-Hot → Word2Vec → Embeddings Contextuais
- **Dimensionalidade**: Como reduzir dimensões mantendo informação
- **Similaridade**: Medindo relações entre palavras no espaço vetorial

### 🔧 Técnicas Aprendidas
- Implementação de embeddings simples
- Visualização com PCA
- Avaliação de qualidade (analogias, similaridade)
- Detecção de bias

### 🚀 Próximos Passos
No **Módulo 6 - Tipos de Modelos**, veremos como diferentes arquiteturas (BERT, GPT, T5) implementam e utilizam embeddings de formas específicas.

**Dica Final do Pedro:** Embeddings são a base de tudo em NLP moderno. Dominar esse conceito é como ter a chave do reino dos LLMs! 🗝️

### 📚 Para Praticar Mais
- Experimente com datasets maiores
- Teste embeddings pré-treinados (Word2Vec do Google, GloVe)
- Explore bibliotecas como Gensim e Hugging Face
- Analise embeddings de diferentes domínios (médico, jurídico, etc.)

---

**Bora para o próximo módulo! 🚀**